<a href="https://colab.research.google.com/github/Mandred009/Indian-Food-Detection-Using-Transfer-Learning/blob/main/Food_Detection_and_Calorie_count_Main_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
import cv2 as cv
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
categories = os.listdir('/content/drive/MyDrive/Freelancing/Indian Food Images/Indian Food Images')

In [ ]:
categories=sorted(categories)

In [ ]:
y=[]

In [ ]:
img_size = 256
def get_data(data_dir):
    data = [] 
    flag=False
    for category in categories: 
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        zz=0
        for img in os.listdir(path):
            try:
                img_arr = cv.imread(os.path.join(path, img)) #[...,::-1] #convert BGR to RGB format #optional
                resized_arr = cv.resize(img_arr, (img_size, img_size)) #reshape
                im_rgb = cv.cvtColor(resized_arr, cv.COLOR_BGR2RGB)
                data.append(im_rgb)
                y.append(class_num)
                zz+=1
                if zz==25:
                  break
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
data=get_data('/content/drive/MyDrive/Freelancing/Indian Food Images/Indian Food Images')

In [ ]:
X=np.array(data)/255.0
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)

In [ ]:
def plot_sample(X, index):
    plt.figure(figsize = (15,2))
    plt.imshow(X[index])
    plt.xlabel(categories[y_train[index]])

In [ ]:
np.shape(y_test)

In [ ]:
ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'
Efficientnet_b0 = "https://tfhub.dev/google/efficientnet/b0/classification/1"
Efficientnet_v2='https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/classification/2'

In [ ]:
model_Efficientnet = tf.keras.Sequential([
    hub.KerasLayer(Efficientnet_v2, trainable = True, input_shape = (256,256,3), name = 'Resnet_V2_50'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(650,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(300,activation='relu'),
    tf.keras.layers.Dense(80, activation = 'softmax', name = 'Output_layer')
])

model_Efficientnet.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [ ]:
efficientnet_model = model_Efficientnet.fit(X_train,y_train, epochs = 160, verbose = 1)

In [ ]:
pred=model_Efficientnet.predict(X_test)

In [ ]:
true_pred=[]
for i in pred:
  true_pred.append(np.argmax(i))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, np.array(true_pred))

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model_Efficientnet.save('/content/drive/MyDrive/Freelancing/Indian Food Images/52Accuracy_zz=25_160ep_650_300')